# Notebook to study the pT distribution of the b-quark from H+->tb for m(H+)~m(top)

use this notebook with setup_atop.sh

In [1]:
import ROOT
%jsroot on
if(not ROOT.xAOD.Init().isSuccess()): print "Failed xAOD.Init()"

Welcome to JupyROOT 6.12/06


In [99]:
treeName = "CollectionTree"
file=ROOT.TFile("output.root","RECREATE")

#get a histogram for a given H+ mass
def gethist(hpmass):
    matchingthreshold=0.4
    
    read_file = "/eos/user/j/jglatzer/lowmasshptruth/daod_truth1_"+str(hpmass)+".root"
    rf = ROOT.TFile.Open(read_file)
    t = ROOT.xAOD.MakeTransientTree(rf, treeName)
    histquark=ROOT.TH1F("bquarkpt"+str(hpmass),"bquarkpt"+str(hpmass),100,0,200) #100 bins between 0 and 200 GeV
    histjet=ROOT.TH1F("bjet"+str(hpmass),"bjetpt"+str(hpmass),100,0,200) #100 bins between 0 and 200 GeV
    total = t.GetEntries()
    for entry in xrange( 0,total): # for entry in [0]:

        t.GetEntry( entry )
        #print entry, t.TruthParticles.size()
        for ni in xrange( t.TruthParticles.size()):
            tr = t.TruthParticles[ni]
            tr_id = tr.pdgId()
            tr_pt = tr.pt()
            tr_eta = tr.eta()
            tr_phi = tr.phi()
            tr_barcode = tr.barcode()
            tr_nChildren = tr.nChildren()
            tr_nParents = tr.nParents()
            barcodes=""
            #print ni, tr_id, tr_pt, tr_barcode
            for ic in xrange( tr_nChildren ):
                if tr.child(ic):
                    barcodes=barcodes+","+str(tr.child(ic).barcode() )
            parentbarcodes=""
            for ic in xrange( tr_nParents ):
                if tr.parent(ic):
                    parentbarcodes=parentbarcodes+","+str(tr.parent(ic).barcode() )+"/"+str(tr.parent(ic).pdgId())
        
            if abs(tr_id)==5 and abs(tr.parent(ic).pdgId())==37:
                histquark.Fill(tr_pt/1000.)
                #now match it to a jet
                quark=ROOT.TVector3()
                quark.SetPtEtaPhi(tr_pt, tr_eta, tr_phi)
                
                mindR=1000000000.
                mindRpt=-20000. #default value
                for nj in xrange( t.AntiKt4TruthJets.size()):
                    tj = t.AntiKt4TruthJets[nj]
                    jet_pt=tj.pt()
                    jet_eta=tj.eta()
                    jet_phi=tj.phi()
                    jet=ROOT.TVector3()
                    jet.SetPtEtaPhi(jet_pt, jet_eta, jet_phi)
                    dR=jet.DeltaR(quark)
                    
                    if dR<matchingthreshold and dR<mindR:
                        mindR=dR
                        mindRpt=jet_pt

                histjet.Fill(mindRpt/1000.)

    return histquark,histjet

In [100]:
cquark={}
cjet={}
histquark={}
histjet={}
def plot(mass):
    histquark[mass],histjet[mass]=gethist(mass)
    cquark[mass] = ROOT.TCanvas("cquark"+str(mass))
    histquark[mass].Draw()
    cquark[mass].Draw()
    cjet[mass] = ROOT.TCanvas("cjet"+str(mass))
    histjet[mass].Draw()
    cjet[mass].Draw()
    bin20quark=histquark[mass].FindBin(20)

    q_eff=0.
    for i in range(bin20quark,histquark[mass].GetNbinsX()+2):
        q_eff+=histquark[mass].GetBinContent(i)
    bin20jet=histjet[mass].FindBin(20)
    j_eff=0.
    for i in range(bin20jet,histjet[mass].GetNbinsX()+2):
        j_eff+=histjet[mass].GetBinContent(i)

    q_eff/=histquark[mass].GetSumOfWeights()
    j_eff/=histjet[mass].GetSumOfWeights()
    print "quark efficiency:", q_eff, "jet efficiency:", j_eff

In [101]:
plot(180)

21.0
11 11
quark efficiency: 0.0452 jet efficiency: 0.378079864061
xAOD::MakeTransientTree   INFO    Created transient tree "CollectionTree" in ROOT's common memory


In [102]:
plot(185)

21.0
11 11
quark efficiency: 0.1504 jet efficiency: 0.318336953149
xAOD::MakeTransientTree   INFO    Created transient tree "CollectionTree" in ROOT's common memory


In [103]:
plot(190)

21.0
11 11
quark efficiency: 0.6256 jet efficiency: 0.53607520564
xAOD::MakeTransientTree   INFO    Created transient tree "CollectionTree" in ROOT's common memory


In [104]:
plot(195)

21.0
11 11
quark efficiency: 0.4914 jet efficiency: 0.450160771704
xAOD::MakeTransientTree   INFO    Created transient tree "CollectionTree" in ROOT's common memory
